![](images/ucl_logo.png)

# Geog0027 Coursework

Prepared by:

[Dr Qingling Wu](http://www.geog.ucl.ac.uk/about-the-department/people/academic-staff/qingling-wu), [Prof Philip Lewis](http://www2.geog.ucl.ac.uk/~plewis/), [Dr Mathias Disney](http://www.geog.ucl.ac.uk/about-the-department/people/academic-staff/mat-disney)

Contact: [Prof Lewis](<mailto: p.lewis@ucl.ac.uk>)

------------------------------------

## Introduction

The Pearl River Delta, and particularly Shenzhen City, in Southern China has been going through a dramatic urbanization process since 1978 due to decentralization policies and market-oriented reforms. 

![encounteringurbanization.files.wordpress.com: Shenzhen 1970](https://encounteringurbanization.files.wordpress.com/2011/06/picture3.png)
[**Shenzhen City 1970**](https://encounteringurbanization.wordpress.com/2011/06/22/shenzhen-the-instant-city/)

![encounteringurbanization.files.wordpress.com: Shenzhen 2011](https://encounteringurbanization.files.wordpress.com/2011/06/picture1.png)
[**Shenzhen City 2011**](https://encounteringurbanization.wordpress.com/2011/06/22/shenzhen-the-instant-city/)

In [6]:
%matplotlib inline
from IPython.display import IFrame
IFrame('http://svs.gsfc.nasa.gov/stories/Landsat/pearl_river.html','100%',490)

In [7]:
IFrame('https://earthengine.google.org/timelapse/player?c=https%3A%2F%2Fearthengine.google.org%2Ftimelapse%2Fdata&v=22.5500,114.1000,8.5&r=.5&p=true"','100%',490)


![encounteringurbanization.files.wordpress.com: Shenzhen 2011](https://encounteringurbanization.files.wordpress.com/2011/06/picture7.png
)
[**Shenzhen Plan of 2005**](https://encounteringurbanization.wordpress.com/2011/06/22/shenzhen-the-instant-city/)



## Purpose of the practical

In this project, we aim to quantify the change of land use due to urbanization, and attempt to explain such change by a number of important socioeconomical drivers. Yearly Landsat TM images are given from 1986 to present. 

## Overview of task

The project is most clearly phrased in two parts:
    
**1. Classification of land cover from a series of Landsat images**

As a first step, we need to quantify at least *three* types of land use for each given year, namely, **urban**, **cropland**, and **other**. In many circumstances, we would want to have cartegories of other as well, but this is not strictly necessary for this practical. Then, we can identify where changes have happened and how much land area has been transformed into urban. See example the land use change map below from [Seto et al. 2002](reading/Setoetal2002.pdf) as anexample of this.
 
![](images/seto.png)

We want to be able to classify quite a large number of images, so we need to bear that in mind when considering what approach to take.
  
A number of classification methods have been taught in this module, and Maximum Likelihood is often a good starting point for multispectral TM imagery. There are also other image processing methods could help to identify land use classes (e.g. vegetation indices, filters, segmentation, etc.). However, if we need to apply training for each scene, the task soon becomes very tedious. We could try to use the same training data for each scene, but this might not work well across multiple sensors.

Instead, we shall use a method of unsupervised classification (ISOdata clustering) to process the bulk of the data. This shifts the problem from one of large manual effort in providing the training datasets to a smaller manual effort making sure we interpret the machine-generated clusters appropriately. This is a basic trade-off of unsupervised versus supervised methods. 

In this work then, you will apply a supervised method and an unsupervised method to *one* scene, then assess the classification accuracy of your approaches. Because of the large number of images involved in the project, we will use an envi program (provided to you) to automate the process of unsupervised classification for the rest of the datasets. 

After you have performed the classifications, you will process each dataset to estimate the area of urban land cover per year. Again, because of the large number of images involved, we will use an envi program to do this.

**2. Modelling Land Use change with socio-economic drivers**

Having derived a dataset of urban land use, we can calculate urban land use change per year.

Following the general approach of [Seto & Kaufmann (2003)](reading/SetoandKaufmann2003.pdf), we will build a multi-linear model to attempt to describe the urban land use change per year (the 'y' variable) as a function of a number of key socioeconomic factors (e.g. capital investment, land productivity, population, wage rates, etc) (the '$x$' variables). 

We suppose that we have observation ('$y$') data for $N_{years}$ and $N_{params}$ of socioeconomic factors for each of these years.

So, we will derive a (multi)*linear* model that we could phrase as:

**Equation 1:**
$$
y = c +  M x                
$$

where we supply the following information:

* $y$ is a column vector of length $N_{years} - 1$ with urban land use change per year. The minus 1 is because we must calculate land use *change* from one year to the next.
* $x$ is an information matrix of size $N_{params} - 1$ by  $N_{years} - 1$ (one entry for each year, for each socioeconomic factor). The minus 1 associated with $N_{params}$ is because we have an additional parameter, $c$.

The model parameters above (i.e. the information we need to derive by fitting to the data) are called $M$ and $c$. Here, $M$ would be a row vector of size $N_{params} - 1$ and $c$ is a scalar. So there are a total of $N_{params}$ *unknown* values (the model parameters) that we need to estimate. 

Following the example in the paper, we will use the following in $x$:

* $x_1$: Investment in capital construction / population
* $x_2$: value of gross agricultural output / agriculture population
* $x_3$: log(wages in non-state, non-collective units)
* $x_4$: log(average total wage)
* $x_5$: (value of gross agricultural output/Agricultural land) / (value of gross industrial output / Urban land)

This will give a model with 6 parameters that we need to estimate (i.e. 6 unknowns) that we could call $p_0, p_1, p_2, p_3, p_4, p_5$. We could then write the general model more specifically as:

**Equation 2:**

$$
y = p_0 + p_1 x_1 + p_2 x_2 + + p_3 x_3 + p_4 x_4 + p_5 x_5
$$

Note that the final term ($x_5$) requires that we have data for Agricultural and Urban land, which we will need to derive from the remote sensing data for each year of observation.

The rest of the data will come from official statistics. An example for the years 1987 to 2000 is given below to give you a feel for the sort of information you will be using:

| Year | Investment in capital construction (100 million yuan) | population (10000) | wages in non-state, non-collective units (100 million yuan) | value of gross agricultural output (100 million yuan) |value of gross industrial output (100 million yuan) | agriculture population (100 million yuan) | average total wage (yuan) | productivity ratio|
|:-:|---|---|---|---|---|---|---|---|
| 1987 | 251.01 | 5832.15 | 4.12 | 348.61 | 878.29 |4522.24 | 1743 |  0.312039409|
| 1988 | 353.59	|5928.31	|7.93|	473.78|	1318.9|	4561.77|	2250|	0.305222663|
|1989 |347.34	|6024.98	|13.27|	548.6|	1647.24|	4602.23|	2678|	0.330817951|
|1990 |381.47	|6246.32	|18.27|	600.71|	1902.25|	4769.01|	2929|	0.380619516|
|1991 |478.2	|6348.95	|28.23|	654.82|	2524.12|	5210.00|	3358|	0.376166347|
|1992 |921.75	|6463.17	|40.28|	737.11|	3479.39|	4822.37|	4027|	0.34788058|
|1993|1629.87|	6581.6	|70.63|	899.03|	5237.37|	4773.51|	5327|	0.349115466|
|1994|2141.15|	6691.46	|103.9|	1151.38|	7273.95|	4726.74|	7117|	0.452607142|
|1995|2327.22|	6788.74	|150.96|	1445.48|	9720.54|	4753.37|	8250|	0.277815958|
|1996|2327.64|	6896.77	|158.93|	1577.89|	10530.93|	4788.97|	9127|	0.508172943|
|1997|2298.14|	7013.73	|198.04|	1656.46|	12372.69|	4840.23|	9698|	0.55606463|
|1998|2668.13|	7115.65	|263.98|	1705.44|	13799.16|	4896.58|	10233|	0.637844818|
|1999|3027.56|	7298.88	|301.44|	1745.02|	15303.33|	5022.46|	11309|	0.749725973|
|2000|3233.7	|   7498.54	|341.98|	1701.18|	16904.47|	5160.25|	13823|	0.814822376	|						


## Summary

The project involves the following tasks:

### Data processing

* Download Landsat annual datasets for 1986 to present (or some suitable subset). **(of secondary importance)**
* For one year (your choice), perform a supervised classification and an unsupervised classification using `envi` and assess the accuracy of the classifications; **(of primary importance)**
* Perform unsupervised classifications (clustering) of the time series of Landsat data, using an `envi` program that you will be provided with;
    * apply suitable class labels, and modify the number of classes as appropriate; **(of primary importance)**
* Calculate the area of urban land use for Shenzhen for each year **(of primary importance)**
* Calculate the area of urban land use change for Shenzhen for each year **(of primary importance)** 
* Estimate the area of agricultural land use for Shenzhen for each year **(of secondary importance)**

### Modelling

* Using the data derived above, calibrate a model that describes urban land use change as a function of a set of socioeconomic factors, following the approach of Seto et al. (2002, 2003) **(of primary importance)**

![](images/comp.png)

* Test the model to assess uncertainty **(of secondary importance)**



## Details

* [Classification](CW-1-PearlRiver-Class.ipynb)
* [Modelling](2021_UrbanModel.ipynb)
* [Write Up](WriteUp.ipynb)



## Further Reading

[Hao, P. (2012) Spatial evolution of urban villages in Shenzhen. PhD thesis University of Twente; Summaries in Dutch and English. ITC Dissertation 205, ISBN: 978-90-6266-295-1.](https://webapps.itc.utwente.nl/librarywww/papers_2012/phd/puhao.pdf)

[Seto, K.C., Woodcock, C.E., Song, C., Huand, X., Lu, J., and Kaufmann, R.K. (2002) Monitoring land-use change in the Pearl River Delta using Landsat TM, Int J. Rem. Sens, 23(10).](reading/Setoetal2002.pdf)

[Seto, K.C., and Kaufmann, R.K., (2003) Modeling the Drivers of Urban Land Use Change in the Pearl River Delta, China: Integrating Remote Sensing with Socioeconomic Data, Land Economics, 79(1): 106-121](reading/SetoandKaufmann2003.pdf)